In [226]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import os

In [227]:
from sklearn.cluster import DBSCAN

In [228]:
os.chdir("/home/vladimir/conda")
df = pd.read_csv("Econom_Cities_data.csv", sep=";",index_col="City",decimal=",")

In [229]:
df[:]

,Work,Price,Salary
City,,,
Amsterdam,1714,65.6,49.0
Athens,1792,53.8,30.4
Bogota,2152,37.9,11.5
Bombay,2052,30.3,5.3
Brussels,1708,73.8,50.5
Buenos_Aires,1971,56.1,12.5
Cairo,-9999,37.1,-9999.0
Caracas,2041,61.0,10.9
Chicago,1924,73.9,61.9


Как видим, у Каира и Джакарты значения двух полей равны $-9999$. Это пропущенные данные, удалим их.

In [230]:
df=df.drop(index=["Cairo","Jakarta"])

Также заметим, что если мы хотим использовать евклидову метрику, то переменная Work будет иметь определяющий вклад. Возьмём значение в Цюрихе за $100$ % и будем использовать проценты, как и в других столбцах.

In [231]:
df["Work"]=df["Work"]/1868*100

In [232]:
dbscan_1 = DBSCAN(eps=1.5, metric='euclidean', min_samples=5)
dbscan_1.fit(df)
dbscan_1.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

 Явно плохо, все наблюдения попали в один кластер. Увеличим eps

In [233]:
dbscan_1 = DBSCAN(eps=15, metric='euclidean', min_samples=5)
dbscan_1.fit(df)
dbscan_1.labels_
unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  8]
 [ 0 19]
 [ 1 19]]


Уже гораздо лучше, но попробуем еще подвигать параметры

In [234]:
dbscan = DBSCAN(eps=15, metric='euclidean', min_samples=2)
dbscan.fit(df)
dbscan.labels_
unique, counts = np.unique(dbscan.labels_, return_counts=True)
df['dbscan'] = dbscan.labels_
print(np.asarray((unique, counts)).T)

[[-1  3]
 [ 0 19]
 [ 1 19]
 [ 2  2]
 [ 3  3]]


Получилось 2 кластера по 19 и два небольших кластера, выбросов всего 3, попробуем сравнить результаты с результатами иерархического кластерного анализа

In [235]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
link = linkage(df, 'ward', 'euclidean')
df['cluster'] = fcluster(link, 4, criterion='maxclust')

In [236]:
tab1 = pd.crosstab(df['dbscan'], df['cluster'])

print(tab1)

cluster   1  2   3  4
dbscan               
-1        0  2   0  1
 0        0  0  17  2
 1       10  9   0  0
 2        0  0   0  2
 3        0  0   0  3


Один кластер почти совпал, но в других разница значительна. В каких индивидах она проявляется? 3 города попали в выбросы

In [237]:
df[df['dbscan']==0]

,Work,Price,Salary,dbscan,cluster
City,,,,,
Amsterdam,91.755889,65.6,49.0,0,3
Brussels,91.434690,73.8,50.5,0,3
Chicago,102.997859,73.9,61.9,0,3
Copenhagen,91.916488,91.3,62.9,0,4
Dublin,94.164882,76.0,41.4,0,3
Dusseldorf,90.631692,78.5,60.2,0,3
Frankfurt,88.329764,74.5,60.4,0,3
Houston,105.888651,71.9,46.3,0,3
London,92.987152,84.2,46.2,0,3


В 0м кластере не совпали только Мадрид и Копенгаген - у них слишком большие цены.

In [238]:
df[df['dbscan']==1].sort_values('cluster')

,Work,Price,Salary,dbscan,cluster
City,,,,,
Manila,121.413276,40.0,4.0,1,1
Bogota,115.203426,37.9,11.5,1,1
Bombay,109.850107,30.3,5.3,1,1
San_Paulo,99.357602,48.9,11.1,1,1
Rio_de_Janeiro,93.629550,46.3,10.5,1,1
Kuala_Lumpur,116.006424,43.5,9.9,1,1
Lagos,95.610278,45.2,2.7,1,1
Panama,111.241970,49.2,13.8,1,1
Mexico_City,104.068522,49.8,5.7,1,1


В первом кластере dbscan попале все города с низкой заплатой, хотя иерархический кластерный анализ разделил их на 2 группы.

In [239]:
df[df['dbscan']==3].sort_values('cluster')

,Work,Price,Salary,dbscan,cluster
City,,,,,
Helsinki,89.239829,113.6,66.6,3,4
Oslo,84.743041,115.5,63.7,3,4
Tokyo,100.642398,115.0,68.0,3,4


Во втором кластере dbscan выделил швейцарские города, а в третьем - города с очень большими ценами, хотя Иерархический КА оставил их в одном кластере.

In [241]:
df[df['dbscan']==-1]

,Work,Price,Salary,dbscan,cluster
City,,,,,
Hong_Kong,127.141328,63.8,27.8,-1,2
Stockholm,96.627409,111.3,39.2,-1,4
Taipei,114.828694,84.3,34.5,-1,2


Гонконг и Тайпей попали в выбосы, как города в которых очень много работают и очень мало зарабатывают.
В Стокгольме слишком высокие цены для таких низких зарплат.